In [ ]:
%load_ext autoreload
%autoreload 2
from airo_drake.stations.ur3e_cart import MakeUR3eCartStation, RunAndPublishSimulation
from airo_drake.planners.planners_base import DualArmPlannerBase

from pydrake.geometry import StartMeshcat

In [ ]:
meshcat = StartMeshcat()

In [ ]:
additional_directives = """- add_model:
        name: brick
        file: package://drake/examples/manipulation_station/models/061_foam_brick.sdf
"""
station = MakeUR3eCartStation(additional_directives)
plant = station.GetSubsystemByName("plant")

In [ ]:
from pydrake.all import PiecewisePose, RigidTransform, RotationMatrix, PiecewisePolynomial
from airo_drake.geometry import top_down_orientation
import numpy as np


class GraspPlanner(DualArmPlannerBase):
    def __init__(self, plant, meshcat):
        super().__init__(plant, meshcat)
        self.planned = False

    def get_openness_trajectory(self, times):
        opened = np.array([self.gripper_max_openness])
        closed = np.array([0.0])
        right_gripper_openness = np.array([self.initial_right_openness_state[0]])
        openness_trajectory = PiecewisePolynomial.FirstOrderHold(
            [times["initial"], times["pregrasp"]], np.hstack([[right_gripper_openness], [opened]])
        )
        openness_trajectory.AppendFirstOrderSegment(times["grasp"], opened)
        openness_trajectory.AppendFirstOrderSegment(times["grasp_close"], closed)
        openness_trajectory.AppendFirstOrderSegment(times["lift"], closed)
        return openness_trajectory

    def Plan(self, context, state):
        if self.planned:
            return
        grasp_pose = self.inital_right_tcp
        pregrasp_pose = self.inital_right_tcp

        global_y_axis = np.array([0, 1, 0])
        grasp_orientation = RotationMatrix(top_down_orientation(gripper_open_direction=global_y_axis))
        grasp_position = np.array([0, 0, 0.02])
        grasp_pose = RigidTransform(grasp_orientation, grasp_position)
        pregrasp_pose = RigidTransform(grasp_orientation, grasp_position + np.array([0, 0, 0.05]))
        lift_pose = RigidTransform(grasp_orientation, grasp_position + np.array([0, 0, 0.2]))

        self.right_tcp_keyposes = {
            "initial": self.inital_right_tcp,
            "pregrasp": pregrasp_pose,
            "grasp": grasp_pose,
            "grasp_close": grasp_pose,
            "lift": lift_pose,
        }
        times = {"initial": 0.0, "pregrasp": 2.5, "grasp": 3.0, "grasp_close": 3.5, "lift": 6.0}

        def MakeLinearTrajectory(keyposes_dict, times_dict):
            keyposes = list(keyposes_dict.values())
            times = list(times_dict.values())
            pose_trajectory = PiecewisePose.MakeLinear(times, keyposes)
            return pose_trajectory

        self.right_tcp_trajectory = MakeLinearTrajectory(self.right_tcp_keyposes, times)
        self.right_openness_trajectory = self.get_openness_trajectory(times)
        self.UpdatePlanVisualization()
        self.planned = True


planner = GraspPlanner(plant, meshcat)


In [ ]:
RunAndPublishSimulation(station, planner, meshcat)